# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

Описание данных:
- children — количество детей в семье
- days_employed — общий трудовой стаж в днях
- dob_years — возраст клиента в годах
- education — уровень образования клиента
- education_id — идентификатор уровня образования
- family_status — семейное положение
- family_status_id — идентификатор семейного положения
- gender — пол клиента
- income_type — тип занятости
- debt — имел ли задолженность по возврату кредитов
- total_income — ежемесячный доход
- purpose — цель получения кредита

# Шаг 1. Откроем файл с данными и изучим общую информацию. 

In [1]:
# <импорт библиотеки pandas>
import pandas as pd

In [2]:
# <чтение файла с данными с сохранением в data_clients>
data_clients = pd.read_csv('/datasets/data.csv')

# <получение первых 10 строк таблицы df>
display(data_clients.head(20))

# <получение общей информации о данных в таблице df>
data_clients.info()


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Количество значений в столбцах различается. Это говорит о том, что в данных есть пропущенные значения (days_employed, total_income).
В данных присутствуют артефакты - отрицательное кол-во дней трудового стажа. 


# Шаг 2. Предобработка данных

### Обработка пропусков

Проверим, отсутствуют ли данные по столбцам days_employed, total_income в одних и тех же строках.

In [3]:
data_clients[(data_clients['days_employed'].isnull() == True) & (data_clients['total_income'].isnull() == True)].info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 2174 entries, 12 to 21510
Data columns (total 12 columns):
children            2174 non-null int64
days_employed       0 non-null float64
dob_years           2174 non-null int64
education           2174 non-null object
education_id        2174 non-null int64
family_status       2174 non-null object
family_status_id    2174 non-null int64
gender              2174 non-null object
income_type         2174 non-null object
debt                2174 non-null int64
total_income        0 non-null float64
purpose             2174 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 220.8+ KB


В строках, где отсутствуют данные по days_employed, также отсутствуют данные по total_income. 
Проверим, из каких профессий эти люди, чтобы понять, можно ли заполнить пропуски средним по этой группе.

In [4]:
data_clients[(data_clients['total_income'].isnull() == True) & (data_clients['days_employed'].isnull() == True)]['income_type'].value_counts()

сотрудник          1105
компаньон           508
пенсионер           413
госслужащий         147
предприниматель       1
Name: income_type, dtype: int64

Профессии оказались разные. Соответственно, нужно применять среднее по каждой группе. 

In [5]:
data_clients['total_income'] = data_clients.groupby('income_type')['total_income'].transform(lambda x: x.fillna(x.mean()))
data_clients['days_employed'] = data_clients.groupby('dob_years')['days_employed'].transform(lambda x: x.fillna(x.mean()))
print(data_clients.head(20))
data_clients.info()

    children  days_employed  dob_years            education  education_id  \
0          1   -8437.673028         42               высшее             0   
1          1   -4024.803754         36              среднее             1   
2          0   -5623.422610         33              Среднее             1   
3          3   -4124.747207         32              среднее             1   
4          0  340266.072047         53              среднее             1   
5          0    -926.185831         27               высшее             0   
6          0   -2879.202052         43               высшее             0   
7          0    -152.779569         50              СРЕДНЕЕ             1   
8          2   -6929.865299         35               ВЫСШЕЕ             0   
9          0   -2188.756445         41              среднее             1   
10         2   -4171.483647         36               высшее             0   
11         0    -792.701887         40              среднее             1   

### Замена типа данных

In [6]:
data_clients['days_employed'] = data_clients['days_employed'].astype('int')

data_clients['total_income'] = data_clients['total_income'].astype('int')
data_clients.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


 Для замены данных на целочисленный использовали метод astype(), так как нам нужен перевод в целое число. 

### Обработка дубликатов

Выводим суммарное количество дубликатов. 

In [7]:
print(data_clients.duplicated().sum())

54


В файле выявили 54 дубликата. Возможные причины появления дубликатов - человеческий фактор.   

In [8]:
data_clients = data_clients.drop_duplicates()
print(data_clients.duplicated().sum())

0


Использовали метод drop_duplicates для удаления дубликатов. Дадее проверили, что дубликатов не осталось.

### Лемматизация

В датафрейме есть столбец purpose, в нем одни и те же причины выражены разными фразами. Сначала выведем список уникальных целей.


In [9]:
unique_purposes = data_clients['purpose'].value_counts()
print(unique_purposes)

свадьба                                   793
на проведение свадьбы                     773
сыграть свадьбу                           769
операции с недвижимостью                  675
покупка коммерческой недвижимости         662
покупка жилья для сдачи                   652
операции с жильем                         652
операции с коммерческой недвижимостью     650
жилье                                     646
покупка жилья                             646
покупка жилья для семьи                   638
строительство собственной недвижимости    635
недвижимость                              633
операции со своей недвижимостью           627
строительство жилой недвижимости          625
покупка недвижимости                      621
покупка своего жилья                      620
строительство недвижимости                619
ремонт жилью                              607
покупка жилой недвижимости                606
на покупку своего автомобиля              505
заняться высшим образованием      

С помощью модуля pymystem3 произведем лемматизацию целей кредита. Для этого будем использовать метод lemmatize().

In [10]:
from pymystem3 import Mystem
m = Mystem() 
list_of_lemmas = []
#purpose = data_clients['purpose'].unique()
for row in data_clients['purpose']:
    lemmas = m.lemmatize(row)
    list_of_lemmas.extend(lemmas)
from collections import Counter
display(Counter(list_of_lemmas))

Counter({'покупка': 5900,
         ' ': 33596,
         'жилье': 4461,
         '\n': 21471,
         'приобретение': 461,
         'автомобиль': 4308,
         'дополнительный': 907,
         'образование': 4014,
         'сыграть': 769,
         'свадьба': 2335,
         'операция': 2604,
         'с': 2918,
         'на': 2228,
         'проведение': 773,
         'для': 1290,
         'семья': 638,
         'недвижимость': 6353,
         'коммерческий': 1312,
         'жилой': 1231,
         'строительство': 1879,
         'собственный': 635,
         'подержать': 478,
         'свой': 2231,
         'со': 627,
         'заниматься': 904,
         'сделка': 941,
         'подержанный': 486,
         'получение': 1315,
         'высокий': 1374,
         'профильный': 436,
         'сдача': 652,
         'ремонт': 607})

Сформируем список топ-5 целей кредита: недвижимость, автомобиль, образование, свадьба, ремонт

### Категоризация данных

Создадим отдельный столбец с категорией кол-ва детей (children_num). Для того, чтобы записать в него значения, используем метод apply().

#### Категоризация по уровню дохода. 
Для определения интервала используем функцию describe().

In [11]:
data_clients['total_income'].describe()

count    2.147100e+04
mean     1.674216e+05
std      9.802416e+04
min      2.066700e+04
25%      1.076545e+05
50%      1.518980e+05
75%      2.024170e+05
max      2.265604e+06
Name: total_income, dtype: float64

In [12]:
def income_level(row):
    if row <= 107654:
                return 'доход низкий'
    if row <= 151898:
                return 'доход средний'
    if row <= 202417:
                return 'доход выше среднего'
    return 'доход высокий' 


Добавим столбец income_level.

In [13]:
data_clients['income_level'] = data_clients['total_income'].apply(income_level)
print(data_clients.head(10)) 

   children  days_employed  dob_years education  education_id  \
0         1          -8437         42    высшее             0   
1         1          -4024         36   среднее             1   
2         0          -5623         33   Среднее             1   
3         3          -4124         32   среднее             1   
4         0         340266         53   среднее             1   
5         0           -926         27    высшее             0   
6         0          -2879         43    высшее             0   
7         0           -152         50   СРЕДНЕЕ             1   
8         2          -6929         35    ВЫСШЕЕ             0   
9         0          -2188         41   среднее             1   

      family_status  family_status_id gender income_type  debt  total_income  \
0   женат / замужем                 0      F   сотрудник     0        253875   
1   женат / замужем                 0      F   сотрудник     0        112080   
2   женат / замужем                 0      M

Создадим словарь по семейным статусам. 
Уточним, сколько раз встречаются разные семейные статусы. 

In [14]:
print(data_clients['family_status'].value_counts())

женат / замужем          12344
гражданский брак          4163
Не женат / не замужем     2810
в разводе                 1195
вдовец / вдова             959
Name: family_status, dtype: int64


Создадим новую таблицу data_clients_dict 

In [15]:
data_clients_dict  = data_clients[['family_status', 'family_status_id']]
print(data_clients_dict.head(10))

      family_status  family_status_id
0   женат / замужем                 0
1   женат / замужем                 0
2   женат / замужем                 0
3   женат / замужем                 0
4  гражданский брак                 1
5  гражданский брак                 1
6   женат / замужем                 0
7   женат / замужем                 0
8  гражданский брак                 1
9   женат / замужем                 0


Удалим дубликаты в таблице data_clients_dict.

In [16]:
data_clients_dict = data_clients_dict.drop_duplicates().reset_index(drop=True)
print(data_clients_dict.sort_values('family_status_id'))

           family_status  family_status_id
0        женат / замужем                 0
1       гражданский брак                 1
2         вдовец / вдова                 2
3              в разводе                 3
4  Не женат / не замужем                 4


# Шаг 3. Ответим на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

Для вывода зависимостей введем функцию relation.

In [17]:
def relation(category):
    return data_clients.groupby(category)['debt'].mean().to_frame().sort_values('debt')

relation('children')

,debt
children,
5,0.000000
-1,0.021277
0,0.075353
3,0.081818
1,0.092327
2,0.094542
4,0.097561
20,0.105263


В столбце children есть неправдоподобные данные (-1 и 20 детей). Также вызывает вопрос нулевая задолженность у заемщиков с 5 детьми. Если это отбросить, прослеживается рост задолженности с увеличеникм кол-ва детей, но у заемщиков с 3-мя детьми задолженность ниже, чем у заемщиков с 1-м ребенком. 
Заемщики, у которых нет детей, меньше всего подвержены образованию задолженности по кредиту. 

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [18]:
relation('family_status')

,debt
family_status,
вдовец / вдова,0.065693
в разводе,0.071130
женат / замужем,0.075421
гражданский брак,0.093202
Не женат / не замужем,0.097509


Лица, не связанные юридическими отношениями брака, имеют самый высокий процент задолженности. 

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [19]:
relation('income_level')

,debt
income_level,
доход высокий,0.070279
доход низкий,0.079545
доход выше среднего,0.084807
доход средний,0.088471


Выведем эту зависимость в сводную таблицу.

In [20]:
data_clients.pivot_table(['debt'],['income_level'],aggfunc='mean', fill_value = 0)


,debt
income_level,
доход высокий,0.070279
доход выше среднего,0.084807
доход низкий,0.079545
доход средний,0.088471


Заемщики с высоким доходом меньше всего склонны к задолженности по кредиту. 

- Как разные цели кредита влияют на его возврат в срок?

In [21]:
def purpose(row):
    if 'недвижим' in row:
                return 'недвижимость'
    if 'жиль' in row:
                return 'недвижимость'
    if 'автомоб' in row:
                return 'автомобиль'
    if 'образов' in row:
                return 'образование'
    if 'свад' in row:
                return 'свадьба'
    if 'ремонт' in row:
                return 'ремонт'
    return 'другое' 

Добавим столбец debt_purpose

In [22]:
data_clients['debt_purpose'] = data_clients['purpose'].apply(purpose)
print(data_clients.head(10)) 

   children  days_employed  dob_years education  education_id  \
0         1          -8437         42    высшее             0   
1         1          -4024         36   среднее             1   
2         0          -5623         33   Среднее             1   
3         3          -4124         32   среднее             1   
4         0         340266         53   среднее             1   
5         0           -926         27    высшее             0   
6         0          -2879         43    высшее             0   
7         0           -152         50   СРЕДНЕЕ             1   
8         2          -6929         35    ВЫСШЕЕ             0   
9         0          -2188         41   среднее             1   

      family_status  family_status_id gender income_type  debt  total_income  \
0   женат / замужем                 0      F   сотрудник     0        253875   
1   женат / замужем                 0      F   сотрудник     0        112080   
2   женат / замужем                 0      M

Выведем зависимость между целями кредита и возвратом кредита в срок

In [23]:
data_clients.pivot_table(['debt'],['debt_purpose'],aggfunc='mean', fill_value = 0)

,debt
debt_purpose,
автомобиль,0.093547
недвижимость,0.072314
образование,0.092177
свадьба,0.079657


Задолженность выше по кредитам, взятым на автомобиль и образование. 

# Шаг 4. Общий вывод

Мы провели исследование, которые включают важнейшие стадии предобработки данных (работа с пропусками, удаление дубликатов, замена типа данных, лемматизация и категоризация данных).   
Для этого мы использовали Python и библиотеки pandas, pymystem3.  
Данное исследование позволило ответить на вопросы, есть ли зависимость между различными характеристиками заемщика и своевременной оплаты по кредиту.   
На своевременную выплату по кредиту могут влиять сразу несколько характеристик заемщика: наличие детей, семейное положение, уровень дохода, цель кредита.   
Наиболее ответственные заемщики - у которых нет детей, кто состоит или состоял в зарегистрированных брачных отношениях, у кого высокий доход, а также те, кто берет кредит на недвижимость или свадьбу. 